In [1]:
import numpy as np
import networkx as nx
import copy
import pickle
import sys
import time
sys.path.append("/home/ubuntu/CE_scheme/")
from BayesCard.Evaluation.utils import parse_query
from Schemas.stats.schema import gen_stats_light_schema
from Join_scheme.data_prepare import identify_key_values, process_stats_data

In [51]:
data_path = "/home/ubuntu/End-to-End-CardEst-Benchmark/datasets/stats_simplified/{}.csv"
schema = gen_stats_light_schema(data_path)
all_keys, equivalent_keys = identify_key_values(schema)
print(equivalent_keys)
model_path = "/home/ubuntu/data_CE/CE_scheme_models/model_stats_sub_optimal_200.pkl"
with open(model_path, "rb") as f:
    new_BE = pickle.load(f)

{'posts.Id': {'comments.PostId', 'postLinks.PostId', 'posts.Id', 'votes.PostId', 'postHistory.PostId', 'tags.ExcerptPostId', 'postLinks.RelatedPostId'}, 'users.Id': {'badges.UserId', 'users.Id', 'comments.UserId', 'postHistory.UserId', 'posts.OwnerUserId', 'votes.UserId'}}


In [42]:
import numpy as np
import copy

from Join_scheme.join_graph import process_condition, get_join_hyper_graph
from Join_scheme.data_prepare import identify_key_values
from BayesCard.Evaluation.cardinality_estimation import timestamp_transorform, construct_table_query


class Factor:
    """
    This the class defines a multidimensional conditional probability.
    """
    def __init__(self, variables, pdfs, equivalent_variables=[]):
        self.variables = variables
        self.equivalent_variables = equivalent_variables
        self.pdfs = pdfs
        self.cardinalities = dict()
        for i, var in enumerate(self.variables):
            self.cardinalities[var] = pdfs.shape[i]
            if len(equivalent_variables) != 0:
                self.cardinalities[equivalent_variables[i]] = pdfs.shape[i]


class Bound_ensemble:
    """
    This the class where we store all the trained models and perform inference on the bound.
    """
    def __init__(self, bns, table_buckets, schema):
        self.bns = bns
        self.table_buckets = table_buckets
        self.schema = schema
        self.all_keys, self.equivalent_keys = identify_key_values(schema)

    def parse_query_simple(self, query):
        """
        If your selection query contains no aggregation and nested sub-queries, you can use this function to parse a
        join query. Otherwise, use parse_query function.
        """
        query = query.replace(" where ", " WHERE ")
        query = query.replace(" from ", " FROM ")
        query = query.replace(" and ", " AND ")
        query = query.split(";")[0]
        query = query.strip()
        tables_all = {}
        join_cond = []
        table_query = {}
        join_keys = {}
        tables_str = query.split(" WHERE ")[0].split(" FROM ")[-1]
        for table_str in tables_str.split(","):
            table_str = table_str.strip()
            if " as " in table_str:
                tables_all[table_str.split(" as ")[-1]] = table_str.split(" as ")[0]
            else:
                tables_all[table_str.split(" ")[-1]] = table_str.split(" ")[0]

        # processing conditions
        conditions = query.split(" WHERE ")[-1].split(" AND ")
        for cond in conditions:
            table, cond, join, join_key = process_condition(cond, tables_all)
            if not join:
                attr = cond[0]
                op = cond[1]
                value = cond[2]
                if "Date" in attr:
                    assert "::timestamp" in value
                    value = timestamp_transorform(value.strip().split("::timestamp")[0])
                if table not in table_query:
                    table_query[table] = dict()
                construct_table_query(self.bns[table], table_query[table], attr, op, value)
            else:
                join_cond.append(cond)
                for tab in join_key:
                    if tab in join_keys:
                        join_keys[tab].add(join_key[tab])
                    else:
                        join_keys[tab] = set([join_key[tab]])

        return tables_all, table_query, join_cond, join_keys

    def get_all_id_conidtional_distribution(self, table_queries, join_keys, equivalent_group):
        res = dict()
        for table in join_keys:
            key_attrs = list(join_keys[table])
            if table in table_queries:
                table_query = table_queries[table]
            else:
                table_query = {}
            id_attrs, probs = self.bns[table].query_id_prob(table_query, key_attrs)
            new_id_attrs = []
            for K in id_attrs:
                for PK in equivalent_group:
                    if K in equivalent_group[PK]:
                        new_id_attrs.append(PK)
            assert len(new_id_attrs) == len(id_attrs)
            res[table] = Factor(id_attrs, probs, new_id_attrs)
        return res

    def eliminate_one_key_group_general(self, tables, key_group, factors):
        rest_groups = dict()
        rest_group_tables = dict()
        for table in tables:
            assert key_group in factors[table].equivalent_variables
            temp = copy.deepcopy(factors[table].equivalent_variables)
            temp.remove(key_group)
            for keys in temp:
                if keys in rest_groups:
                    assert factors[table].cardinalities[keys] == rest_groups[keys]
                    rest_group_tables[keys].append(table)
                else:
                    rest_groups[keys] = factors[table].cardinalities[keys]
                    rest_group_tables[keys] = [table]

        new_factor_variables = []
        new_factor_cardinalities = []
        for key in rest_groups:
            new_factor_variables.append(key)
            new_factor_cardinalities.append(rest_groups[key])
        new_factor_pdf = np.zeros(tuple(new_factor_cardinalities))

    def eliminate_one_key_group(self, tables, key_group, factors, relevant_keys):
        """This version only supports 2D distributions"""
        #print(key_group)
        rest_group = None
        rest_group_cardinalty = 0
        eliminated_tables = []
        rest_group_tables = []
        for table in tables:
            assert key_group in factors[table].equivalent_variables
            temp = copy.deepcopy(factors[table].equivalent_variables)
            temp.remove(key_group)
            if len(temp) == 0:
                eliminated_tables.append(table)
            for key in temp:
                if rest_group:
                    assert factors[table].cardinalities[key] == rest_group_cardinalty
                    rest_group_tables.append(table)
                else:
                    rest_group = key
                    rest_group_cardinalty = factors[table].cardinalities[key]
                    rest_group_tables = [table]

        all_probs_eliminated = []
        all_modes_eliminated = []
        #print(eliminated_tables)
        #print(rest_group_tables)
        for table in eliminated_tables:
            bin_modes = self.table_buckets[table].oned_bin_modes[relevant_keys[key_group][table]]
            all_probs_eliminated.append(factors[table].pdfs)
            all_modes_eliminated.append(np.minimum(bin_modes, factors[table].pdfs))
        if rest_group:
            new_factor_pdf = np.zeros(rest_group_cardinalty)
        else:
            return self.compute_bound_oned(all_probs_eliminated, all_modes_eliminated)
        
        #print(np.sum(factors[rest_group_tables[0]].pdfs))
        #print(np.sum(factors[rest_group_tables[0]].pdfs, axis=1))
        
        #print(self.table_buckets[rest_group_tables[0]].twod_bin_modes[relevant_keys[key_group][rest_group_tables[0]]])
        for i in range(rest_group_cardinalty):
            #print(i)
            rest_group_probs_eliminated = []
            rest_group_modes_eliminated = []
            for table in rest_group_tables:
                
                idx_f = factors[table].equivalent_variables.index(key_group)
                idx_b = self.table_buckets[table].id_attributes.index(relevant_keys[key_group][table])
                #print(idx_f, idx_b)
                bin_modes = self.table_buckets[table].twod_bin_modes[relevant_keys[key_group][table]]
                if idx_f == 0 and idx_b == 0:
                    rest_group_probs_eliminated.append(factors[table].pdfs[:, i])
                    rest_group_modes_eliminated.append(np.minimum(bin_modes[:, i], factors[table].pdfs[:, i]))
                elif idx_f == 0 and idx_b == 1:
                    rest_group_probs_eliminated.append(factors[table].pdfs[:, i])
                    rest_group_modes_eliminated.append(np.minimum(bin_modes[i, :], factors[table].pdfs[:, i]))
                elif idx_f == 1 and idx_b == 0:
                    #print(np.sum(factors[table].pdfs[i, :]))
                    rest_group_probs_eliminated.append(factors[table].pdfs[i, :])
                    rest_group_modes_eliminated.append(np.minimum(bin_modes[:, i], factors[table].pdfs[i, :]))
                else:
                    rest_group_probs_eliminated.append(factors[table].pdfs[i, :])
                    rest_group_modes_eliminated.append(np.minimum(bin_modes[i, :], factors[table].pdfs[i, :]))
            #print("non-zero prob:", len(all_probs_eliminated), np.sum(all_probs_eliminated[-1] != 0), np.sum(all_probs_eliminated[-1]))
            #print("non-zero mode:", len(all_modes_eliminated), np.sum(all_modes_eliminated[-1] != 0), np.sum(all_modes_eliminated[-1]))
            new_factor_pdf[i] = self.compute_bound_oned(all_probs_eliminated + rest_group_probs_eliminated, 
                                                        all_modes_eliminated + rest_group_modes_eliminated)

        for table in rest_group_tables:
            factors[table] = Factor([rest_group], new_factor_pdf, [rest_group])

        return None

    def compute_bound_oned(self, all_probs, all_modes):
        all_probs = np.stack(all_probs, axis=0)
        #print(np.sum(all_probs != 0))
        all_modes = np.stack(all_modes, axis=0)
        #print(np.sum(all_modes != 0))
        multiplier = np.prod(all_modes, axis=0)
        non_zero_idx = np.where(multiplier != 0)[0]
        #print(len(non_zero_idx))
        min_number = np.amin(all_probs[:, non_zero_idx]/all_modes[:, non_zero_idx], axis=0)
        multiplier[non_zero_idx] = multiplier[non_zero_idx] * min_number
        #print("multipler:", np.sum(multiplier))
        return np.sum(multiplier)

    def get_optimal_elimination_order(self, equivalent_group, join_keys, factors):
        cardinalities = dict()
        lengths = dict()
        tables_involved = dict()
        relevant_keys = dict()
        for group in equivalent_group:
            relevant_keys[group] = dict()
            lengths[group] = len(equivalent_group[group])
            cardinalities[group] = []
            tables_involved[group] = set([])
            for keys in equivalent_group[group]:
                for table in join_keys:
                    if keys in join_keys[table]:
                        cardinalities[group].append(len(join_keys[table]))
                        tables_involved[group].add(table)
                        variables = factors[table].variables
                        variables[variables.index(keys)] = group
                        factors[table].variables = variables
                        relevant_keys[group][table] = keys
                        break
            cardinalities[group] = np.asarray(cardinalities[group])

        optimal_order = list(equivalent_group.keys())
        for i in range(len(optimal_order)):
            min_idx = i
            for j in range(i+1, len(optimal_order)):
                min_group = optimal_order[min_idx]
                curr_group = optimal_order[j]
                if np.max(cardinalities[curr_group]) < np.max(cardinalities[min_group]):
                    min_idx = j
                else:
                    min_max_tables = np.max(cardinalities[min_group])
                    min_num_max_tables = len(np.where(cardinalities[min_group] == min_max_tables)[0])
                    curr_max_tables = np.max(cardinalities[curr_group])
                    curr_num_max_tables = len(np.where(cardinalities[curr_group] == curr_max_tables)[0])
                    if curr_num_max_tables < min_num_max_tables:
                        min_idx = j
                    elif lengths[curr_group] < lengths[min_group]:
                        min_idx = j
            optimal_order[i], optimal_order[min_idx] = optimal_order[min_idx], optimal_order[i]
        return optimal_order, tables_involved, relevant_keys

    def get_cardinality_bound(self, query_str):
        tables_all, table_queries, join_cond, join_keys = self.parse_query_simple(query_str)
        equivalent_group = get_join_hyper_graph(join_keys, self.equivalent_keys)
        conditional_factors = self.get_all_id_conidtional_distribution(table_queries, join_keys, equivalent_group)
        optimal_order, tables_involved, relevant_keys = self.get_optimal_elimination_order(equivalent_group, join_keys,
                                                                            conditional_factors)

        for key_group in optimal_order:
            tables = tables_involved[key_group]
            res = self.eliminate_one_key_group(tables, key_group, conditional_factors, relevant_keys)
        return res


In [52]:
BE = Bound_ensemble(new_BE.bns, new_BE.table_buckets, new_BE.schema)

In [53]:
query57 = "SELECT COUNT(*) FROM posts as p, postLinks as pl, postHistory as ph, votes as v, badges as b, users as u WHERE p.Id = pl.RelatedPostId AND u.Id = p.OwnerUserId AND u.Id = b.UserId AND u.Id = ph.UserId AND u.Id = v.UserId AND p.CommentCount>=0 AND p.CommentCount<=13 AND ph.PostHistoryTypeId=5 AND ph.CreationDate<='2014-08-13 09:20:10'::timestamp AND v.CreationDate>='2010-07-19 00:00:00'::timestamp AND b.Date<='2014-09-09 10:24:35'::timestamp AND u.Views>=0 AND u.DownVotes>=0 AND u.CreationDate>='2010-08-04 16:59:53'::timestamp AND u.CreationDate<='2014-07-22 15:15:22'::timestamp;"
t = time.time()
res = BE.get_cardinality_bound(query57)
print(time.time() - t)
print(res)

0.0678858757019043
1167709503258.581


In [45]:
query_file = "/home/ubuntu/End-to-End-CardEst-Benchmark/workloads/stats_CEB/sub_plan_queries/stats_CEB_sub_queries.sql"
with open(query_file, "r") as f:
    queries = f.readlines()


In [54]:
qerror = []
latency = []
pred = []
for i, query_str in enumerate(queries):
    query = query_str.split("||")[0][:-1]
    print("========================")
    true_card = int(query_str.split("||")[-1])
    t = time.time()
    res = BE.get_cardinality_bound(query)
    pred.append(res)
    latency.append(time.time() - t)
    qerror.append(res/true_card)
    print(f"estimating query {i}: predicted {res}, true_card {true_card}, qerror {res/true_card}, latency {time.time() - t}")

estimating query 0: predicted 79851.0, true_card 79851, qerror 1.0, latency 0.006472110748291016
estimating query 1: predicted 12029558.0, true_card 10220614, qerror 1.1769897581495594, latency 0.007853269577026367
estimating query 2: predicted 3936570.0, true_card 1458075, qerror 2.699840543181935, latency 0.00991678237915039
estimating query 3: predicted 4018726.3489689557, true_card 1709781, qerror 2.3504333882344906, latency 0.011612176895141602
estimating query 4: predicted 10535179.0, true_card 7491903, qerror 1.4062086762201806, latency 0.010989665985107422
estimating query 5: predicted 1028906.4177326594, true_card 428612, qerror 2.4005543888940566, latency 0.011984825134277344
estimating query 6: predicted 59711027.36463124, true_card 55900138, qerror 1.068173165594533, latency 0.011457204818725586
estimating query 7: predicted 11102.0, true_card 10972, qerror 1.0118483412322274, latency 0.008883237838745117
estimating query 8: predicted 60665445.0, true_card 699302, qerror 86

estimating query 59: predicted 33875.14264761277, true_card 26836, qerror 1.2623022301241902, latency 0.028447628021240234
estimating query 60: predicted 3682949.7701961263, true_card 2704241, qerror 1.3619162530987905, latency 0.006556510925292969
estimating query 61: predicted 34466.642735042726, true_card 32918, qerror 1.047045468589912, latency 0.005743265151977539
estimating query 62: predicted 88199.77975695922, true_card 86112, qerror 1.0242449340040787, latency 0.006495475769042969
estimating query 63: predicted 3419123.514235027, true_card 2488080, qerror 1.3742015989176501, latency 0.007988929748535156
estimating query 64: predicted 1596558.0, true_card 1056687, qerror 1.510909096071022, latency 0.003872394561767578
estimating query 65: predicted 31467.393518104083, true_card 20334, qerror 1.547525991841452, latency 0.005399942398071289
estimating query 66: predicted 74059.51791287246, true_card 62294, qerror 1.1888708047785093, latency 0.004387378692626953
estimating query 6

estimating query 138: predicted 3279394.53128791, true_card 298796, qerror 10.975362894041119, latency 0.008033514022827148
estimating query 139: predicted 339541802.5951356, true_card 6672465, qerror 50.88701141109554, latency 0.01325678825378418
estimating query 140: predicted 236895413.90759227, true_card 235010920, qerror 1.008018750395055, latency 0.0047876834869384766
estimating query 141: predicted 13685912.524602357, true_card 10967392, qerror 1.2478730152621842, latency 0.03125143051147461
estimating query 142: predicted 95834.92233764203, true_card 64129, qerror 1.4944084944041234, latency 0.007779121398925781
estimating query 143: predicted 12950459.231511563, true_card 12470361, qerror 1.0384991446126992, latency 0.031500816345214844
estimating query 144: predicted 159107.35384901142, true_card 140071, qerror 1.135905032797734, latency 0.00789332389831543
estimating query 145: predicted 30521.51415098377, true_card 20007, qerror 1.5255417679304129, latency 0.033969163894653

estimating query 208: predicted 129792.81583936054, true_card 130707, qerror 0.993005851556233, latency 0.004070281982421875
estimating query 209: predicted 971806.1618962524, true_card 443440, qerror 2.1915166919904663, latency 0.0048980712890625
estimating query 210: predicted 41392.49881996187, true_card 42002, qerror 0.9854887581534657, latency 0.003676176071166992
estimating query 211: predicted 78348.05250682085, true_card 79006, qerror 0.9916721832116656, latency 0.0027663707733154297
estimating query 212: predicted 268218798.31406698, true_card 105742741, qerror 2.5365220891528333, latency 0.007543087005615234
estimating query 213: predicted 3862363.7504616203, true_card 1423047, qerror 2.7141505167866, latency 0.005523681640625
estimating query 214: predicted 11784323.28323553, true_card 9976404, qerror 1.181219533935828, latency 0.005581378936767578
estimating query 215: predicted 971806.1618962524, true_card 443440, qerror 2.1915166919904663, latency 0.004864692687988281
est

estimating query 292: predicted 201981.33270332383, true_card 55293, qerror 3.6529277250885976, latency 0.006292819976806641
estimating query 293: predicted 69860.27914396262, true_card 66311, qerror 1.053524741656175, latency 0.00783991813659668
estimating query 294: predicted 1344065.4604461123, true_card 1228382, qerror 1.0941754767215022, latency 0.009340763092041016
estimating query 295: predicted 289474.6340988182, true_card 10594, qerror 27.32439438350181, latency 0.007925033569335938
estimating query 296: predicted 3307472.788772854, true_card 160411, qerror 20.61874054006804, latency 0.008213520050048828
estimating query 297: predicted 689139.8684369789, true_card 147586, qerror 4.669412196529338, latency 0.010567426681518555
estimating query 298: predicted 6158731.144236522, true_card 194687, qerror 31.634013284074037, latency 0.007197141647338867
estimating query 299: predicted 1698251.7994525228, true_card 262343, qerror 6.47340237571623, latency 0.009523630142211914
estima

...

estimating query 2343: predicted 338585.7676307812, true_card 303416, qerror 1.1159126994976574, latency 0.03481793403625488
estimating query 2344: predicted 1394538.1799698793, true_card 1326566, qerror 1.0512391995346475, latency 0.032111167907714844
estimating query 2345: predicted 79851.0, true_card 79851, qerror 1.0, latency 0.0011036396026611328
estimating query 2346: predicted 277274.36368926027, true_card 18316, qerror 15.138368840863741, latency 0.007635593414306641
estimating query 2347: predicted 2762086.851194438, true_card 361425, qerror 7.642213048888256, latency 0.008804082870483398
estimating query 2348: predicted 711236.0622184487, true_card 253217, qerror 2.8088006027180192, latency 0.03339743614196777
estimating query 2349: predicted 11916163.865151381, true_card 5543342, qerror 2.1496353400442154, latency 0.012987613677978516
estimating query 2350: predicted 129488.35108064536, true_card 63198, qerror 2.0489311541606594, latency 0.012590408325195312
estimating query

estimating query 2399: predicted 6430387.248712353, true_card 44869387, qerror 0.14331346333553327, latency 0.03825092315673828
estimating query 2400: predicted 38941615.53990964, true_card 226229172, qerror 0.17213348391652, latency 0.03885221481323242
estimating query 2401: predicted 113996946.489083, true_card 299574955, qerror 0.3805289614049446, latency 0.04188394546508789
estimating query 2402: predicted 79850947.85011286, true_card 3596066, qerror 22.205084069678602, latency 0.041113853454589844
estimating query 2403: predicted 5726574.985902067, true_card 11214638, qerror 0.510633957681208, latency 0.013942956924438477
estimating query 2404: predicted 6430387.248712353, true_card 42793812, qerror 0.15026441787219968, latency 0.03893566131591797
estimating query 2405: predicted 38941615.53990964, true_card 139632376, qerror 0.27888672137119286, latency 0.041962623596191406
estimating query 2406: predicted 113996946.489083, true_card 319981016, qerror 0.35626159299738897, latency

estimating query 2461: predicted 105779768.97592029, true_card 1328499, qerror 79.62352171580129, latency 0.014326333999633789
estimating query 2462: predicted 5388569.325133598, true_card 12492036, qerror 0.4313603743323825, latency 0.013574361801147461
estimating query 2463: predicted 5236263.015668884, true_card 113409, qerror 46.17149446400977, latency 0.011312723159790039
estimating query 2464: predicted 6110946.0033638235, true_card 21582838, qerror 0.2831391313488904, latency 0.017461061477661133
estimating query 2465: predicted 4692940.180863893, true_card 182241, qerror 25.75128637827872, latency 0.015476465225219727
estimating query 2466: predicted 18829555.89622193, true_card 1785176, qerror 10.547730809859605, latency 0.01463770866394043
estimating query 2467: predicted 35493429.06422247, true_card 96498890, qerror 0.36781178585808055, latency 0.017841100692749023
estimating query 2468: predicted 16829528.569834143, true_card 975631, qerror 17.24989116769982, latency 0.0161

estimating query 2528: predicted 70142.96946858056, true_card 64243, qerror 1.0918383243089607, latency 0.0074694156646728516
estimating query 2529: predicted 331310.2141987304, true_card 357171, qerror 0.927595505230633, latency 0.008491992950439453
estimating query 2530: predicted 322886.15614350786, true_card 33758, qerror 9.564730023801998, latency 0.00596165657043457
estimating query 2531: predicted 2142007.409825555, true_card 117788, qerror 18.185277021645287, latency 0.006409168243408203
estimating query 2532: predicted 953605.3297434251, true_card 412822, qerror 2.3099673218564543, latency 0.009360790252685547
estimating query 2533: predicted 185670.30230000033, true_card 88722, qerror 2.0927199826424148, latency 0.014618158340454102
estimating query 2534: predicted 6632378.4381467365, true_card 51426, qerror 128.96936254320258, latency 0.004633426666259766
estimating query 2535: predicted 1746522.8101777385, true_card 265135, qerror 6.587296321412634, latency 0.00745677947998

estimating query 2588: predicted 320659.10374243197, true_card 32154, qerror 9.972603835990295, latency 0.015781641006469727
estimating query 2589: predicted 18621755.43964935, true_card 3482755, qerror 5.3468462293929235, latency 0.009430408477783203
estimating query 2590: predicted 3562569.460304135, true_card 649110, qerror 5.4883909665605755, latency 0.015544414520263672
estimating query 2591: predicted 884226.7953301966, true_card 790217, qerror 1.1189670626298809, latency 0.012958765029907227
estimating query 2592: predicted 36223079.11286827, true_card 1706906, qerror 21.221484436089785, latency 0.007252931594848633
estimating query 2593: predicted 1388078.7423824386, true_card 387888, qerror 3.5785555170111953, latency 0.009064674377441406
estimating query 2594: predicted 217559.2812562054, true_card 57968, qerror 3.7530927624931927, latency 0.015431880950927734
estimating query 2595: predicted 66586.89430284835, true_card 64156, qerror 1.0378903657155738, latency 0.01314735412

In [55]:
qerror = np.asarray(qerror)

In [56]:
temp_qerror = copy.deepcopy(qerror)
temp_qerror[temp_qerror < 1] = 1/temp_qerror[temp_qerror < 1]

In [57]:
for i in [50, 90, 95, 99, 100]:
    print(f"q-error {i}% percentile is {np.percentile(temp_qerror, i)}")

q-error 50% percentile is 3.269330099670447
q-error 90% percentile is 45.854624152810196
q-error 95% percentile is 146.14350723707727
q-error 99% percentile is 1371.2066796610022
q-error 100% percentile is 438398.8024339036


In [50]:
for i in [50, 90, 95, 99, 100]:
    print(f"q-error {i}% percentile is {np.percentile(temp_qerror, i)}")

q-error 50% percentile is 2.5953660431994563
q-error 90% percentile is 32.120346872131535
q-error 95% percentile is 84.13650230255473
q-error 99% percentile is 1072.511697880098
q-error 100% percentile is 981187.002587944


In [ ]:
with open("stats_CEB_sub_queries_CE_scheme.txt", "w") as f:
    for p in pred:
        f.write(str(p)+"\n")

In [ ]:
with open("stats_CEB_exec.sql", "r") as f:
    queries = f.readlines()

In [ ]:
with open("stats_CEB_exec.sql", "w") as f:
    for q in queries:
        q = q.split("||")[-1]
        f.write(q)